# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.oncvpsp3.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397837979015                   -0.89    5.0   36.6ms
  2   -8.400254931068       -2.62       -1.73    1.0   25.1ms
  3   -8.400404100895       -3.83       -2.93    1.5   26.6ms
  4   -8.400428652102       -4.61       -2.91    3.5   56.4ms
  5   -8.400428803824       -6.82       -3.02    1.0   25.1ms
  6   -8.400429017876       -6.67       -4.69    1.0   25.1ms
  7   -8.400429023972       -8.21       -4.42    2.5   31.4ms
  8   -8.400429024395       -9.37       -5.13    1.0   25.2ms
  9   -8.400429024418      -10.64       -6.24    1.0   40.9ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397881881896                   -0.90           5.2    498ms
  2   -8.400384637862       -2.60       -1.79   0.80    2.2    503ms
  3   -8.400423533878       -4.41       -3.02   0.80    1.0    177ms
  4   -8.400428979680       -5.26       -3.44   0.80    2.5   70.6ms
  5   -8.400429021281       -7.38       -4.71   0.80    1.2   22.9ms
  6   -8.400429024401       -8.51       -5.63   0.80    3.2   30.1ms
  7   -8.400429024420      -10.73       -6.30   0.80    2.0   25.7ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +0.511929385711                   -1.05    2.84s
  2   -1.899768454959        0.38       -0.66    126ms
  3   -3.851191192473        0.29       -0.40   57.5ms
  4   -4.901628258454        0.02       -0.44   57.1ms
  5   -6.720143537204        0.26       -0.56   82.0ms
  6   -7.584697822042       -0.06       -0.82   56.7ms
  7   -7.770202672421       -0.73       -1.42   42.1ms
  8   -8.065475715411       -0.53       -1.69   42.0ms
  9   -8.136478001173       -1.15       -1.88   58.8ms
 10   -8.261526126942       -0.90       -1.93   42.6ms
 11   -8.301802015749       -1.39       -2.14   42.5ms
 12   -8.324686379806       -1.64       -2.27   42.4ms
 13   -8.348319200765       -1.63       -2.15   42.1ms
 14   -8.368382257007       -1.70       -2.36   46.1ms
 15   -8.384161044771       -1.80       -2.37   42.1ms
 16   -8.392338982169       -2.09       -3.20   42.1ms
 17   -8.39

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397754038464                   -0.90    5.0   50.0ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400428849620                   -1.78    25.2s
  2   -8.400429024420       -6.76       -4.03    2.41s
  3   -8.400429024420      -14.45       -7.82    151ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.0575067410721812e-6
|ρ_newton - ρ_scfv| = 3.6519829031862675e-7
|ρ_newton - ρ_dm|   = 3.3273158834762693e-6
